In [19]:
# Specify your image_path right here
url = 'https://zenherald.com/wp-content/uploads/2020/10/Modern-day-Viking-e1618564239690.webp'

In [16]:
#@title Run this cell to download necessary codes - Only need to run once
%%capture
!pip install transformers
import torch
import torch.nn as nn
from PIL import Image
import torchvision as tv
import numpy as np
from transformers import BertTokenizer
import requests
model = torch.hub.load('saahiluppal/catr', 'v3', pretrained=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
start_token = tokenizer.convert_tokens_to_ids(tokenizer._cls_token)
end_token = tokenizer.convert_tokens_to_ids(tokenizer._sep_token)

def under_max(image):
    if image.mode != 'RGB':
        image = image.convert("RGB")

    shape = np.array(image.size, dtype=np.float)
    long_dim = max(shape)
    scale = 299 / long_dim

    new_shape = (shape * scale).astype(int)
    image = image.resize(new_shape)

    return image

val_transform = tv.transforms.Compose([
    tv.transforms.Lambda(under_max),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def create_caption_and_mask(start_token, max_length):
    caption_template = torch.zeros((1, max_length), dtype=torch.long)
    mask_template = torch.ones((1, max_length), dtype=torch.bool)

    caption_template[:, 0] = start_token
    mask_template[:, 0] = False

    return caption_template, mask_template
  
caption, cap_mask = create_caption_and_mask(start_token, 128)

In [20]:
#@title Run this cell to caption image. Rerun this cell if you change the image url
image = Image.open(requests.get(url, stream=True).raw)
image = val_transform(image)
image = image.unsqueeze(0)
@torch.no_grad()
def evaluate():
    model.eval()
    for i in range(128 - 1):
        predictions = model(image, caption, cap_mask)
        predictions = predictions[:, i, :]
        predicted_id = torch.argmax(predictions, axis=-1)

        if predicted_id[0] == 102:
            return caption

        caption[:, i+1] = predicted_id[0]
        cap_mask[:, i+1] = False

    return caption

output = evaluate()
result = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
print(result.capitalize())

A man with a mask on his head is holding a horse.
